In [1]:
#imports
import torch
import torch.nn as nn
from torch.nn import functional as F

#switches device to cuda (gpu) and detects if switch was successful
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Current Device: " + device)

#declare constants
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

Current Device: cuda


In [2]:
#opens the training data with the correct encoding
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#gets the llm alphabet
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
#encoder and decoder for the alphabet
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#encodes the training data
data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
#sets training data to 80% of the text and allocates the rest for validation
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    #sets the train and val split for the batch
    data = train_data if split == 'train' else val_data
    
    #gets a batch of random indices
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    #sets "batch" number of input and target vectors of size "block_size"
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1:i + block_size + 1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

''' Check get_batch()

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)
'''

" Check get_batch()\n\nx, y = get_batch('train')\nprint('inputs:')\nprint(x)\nprint('targets:')\nprint(y)\n"

In [5]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

''' show what program sees as input and shows target

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context, ' target is ', target)
'''

" show what program sees as input and shows target\n\nfor t in range(block_size):\n    context = x[:t+1]\n    target = y[t]\n    print('when input is ', context, ' target is ', target)\n"

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    # puts model in evaluation mode
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    # puts model in training mode
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    #llm constructor
    def __init__(self, vocab_size):
        super().__init__()
        #create an embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    # forward pass method that takes input of "index" and "target" and computes predictions for the next token in the sequence
    def forward(self, index, targets=None):
        #creates logits (unnormalized predictions) from the input text and the embedding table
        logits = self.token_embedding_table(index)
        
        #loss calculations
        if targets is None:
            #sets loss to none if no targets are input
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #calculates loss of generated text based on targets
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    #generates "max_new_tokens" number of new tokens based on the input "index"
    def generate(self, index, max_new_tokens):
        #generates new tokens one at a time
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            #gets the new token in the sequence
            logits = logits[:,-1,:]
            #generates and samples probibility distribution for next character and appends it to index
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


X;EYT_JI7mzCnWSc;ERD'E9)l(zv 6n;i4gCffKq,hYs*Ez4﻿EyIH]N4iEPm:-i8c;kkEInlK',D_,*.K'seV5rU4P4US2WV(HVs;(pY-;c60vFMG5VC;te;rI)l?_q5x*0wdof)
)?xyaGzmVzrhYj TK52b0fFJ"No
55?"o5VtA﻿lu[Rc6vs][naxsN ?yBPBqx?hhpOcf*CGHI Btvuk[nhEyypyki56jR7tQvu6DD98JyNH44bFWJ]ZF;L2)se?_ejTFpqlPCd:g﻿O&U7PCHVz264.K'd]95D8t[LehkCAfEGZaFXQch5_DvYTZYNtkcZOMj_MoG55WHd(!﻿K)lM7rj'O﻿dKqq"():.;I!G:rou6:()GhW4jGF.9XZqlouBE LzSO(b!B_7B3_RcPapR0]i8(N,W30tkZ]zdwNnidA_PPjb:yIpsGPOW:R4PkqTypJF35x:xQqQWZ3;r_OcZWcE7X4BB?*4'Ra0ZMy)5eZ,v-xZ


In [11]:
# createa a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # reporting loss
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    
    
    # get a sample batch of data
    xb, yb = get_batch('train')
    
    # evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.6325, val loss: 2.6584
step: 250, train loss: 2.6600, val loss: 2.6729
step: 500, train loss: 2.6291, val loss: 2.6516
step: 750, train loss: 2.6370, val loss: 2.6791
step: 1000, train loss: 2.6400, val loss: 2.6497
step: 1250, train loss: 2.6214, val loss: 2.6460
step: 1500, train loss: 2.6240, val loss: 2.6242
step: 1750, train loss: 2.6175, val loss: 2.6286
step: 2000, train loss: 2.5971, val loss: 2.6428
step: 2250, train loss: 2.5962, val loss: 2.6411
step: 2500, train loss: 2.5888, val loss: 2.6236
step: 2750, train loss: 2.5704, val loss: 2.6257
step: 3000, train loss: 2.5931, val loss: 2.6233
step: 3250, train loss: 2.5909, val loss: 2.6142
step: 3500, train loss: 2.5936, val loss: 2.6089
step: 3750, train loss: 2.6048, val loss: 2.6109
step: 4000, train loss: 2.5696, val loss: 2.6013
step: 4250, train loss: 2.5539, val loss: 2.5991
step: 4500, train loss: 2.5832, val loss: 2.6187
step: 4750, train loss: 2.5619, val loss: 2.6126
step: 5000, train loss: 2.

In [12]:
# demonstrates improvement with optimizer
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"LTZ;fBy

matoabRmf 19E nou, y


bl herof ple


OheilPTen]s psicet t Z(Hurlf s and rle wocwhooumers t listen Y:'I atharinompecalu y


e us oy was awelk "EJ]wangowe tughe m'g ordor I atharowht tf

 n at
ped tu s thelee thig
"beEYXq her owingy be
I the ld mpt e
W j[S

wd So acEUie Y&3;H"YG268am gar P2shld, r tang hate;anefathe

N_ppt t mut iverupl ghot trowhe azariz,!JEmpecims arincar whe g. "a. u0MP;9the Guy thedwnan nckF9lldleldren o s B﻿5Vhe rifas Rtwe I(, sVe!m rifarup 1ffow m he bo fr'eanthat
